In [1]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

from pyomo.core.util import quicksum

import pandas as pd
from datetime import date
import itertools

In [12]:
money_in = pd.DataFrame([[1, 0, 10],
                         [2, 0, 10],
                         [1, 1, 20],
                         [2, 1, 20]]
                         , columns=['TID', 'date', 'money_in'])

TIDS = money_in['TID'].unique()
DATES = list(range(len(money_in['date'].unique())))

quantity_cars = 6

kwargs = {}

M = 100000000000

MAX_MONEY = 1000000

MAX_DAYS_INC = 14

OVERNIGHT = 2

days_from_inc_dict = {tid: 0 for tid in TIDS}

money_in_dict = money_in.set_index(['TID', 'date']).to_dict()['money_in']

money_in_dict

DATES

[0, 1]

{(1, 0): 10, (2, 0): 10, (1, 1): 20, (2, 1): 20}

In [13]:

# BIG_NUM = 40*60
# postomat_places = list(fixed_points) + list(possible_postomats)
# center_mass_set = set(distances['id_center_mass'].to_list())
# metro_points_set = set(distanses_metro['object_id_metro'].to_list())

# distances_dict = {(id_center_mass, postomat_place_id): walk_time for _, postomat_place_id, id_center_mass , _, _, walk_time in distances.itertuples()}
# distances_metro_dict = {(object_id_metro, object_id): walk_time for _, object_id, object_id_metro ,  _, walk_time in distanses_metro.itertuples()}



# Создание конкретной модели pyomo
model = pyo.ConcreteModel()

model.MAX_MONEY = MAX_MONEY

model.MAX_DAYS_INC = MAX_DAYS_INC

model.OVERNIGHT_BY_DAY = OVERNIGHT/100/365

model.days_from_inc_dict = days_from_inc_dict

model.money_in_dict = money_in_dict

# Переменные
model.money_inc = pyo.Var(TIDS, DATES, within=pyo.Binary, initialize=0)


model.money_inside_TID = pyo.Var(TIDS, DATES,  within=pyo.NonNegativeReals, initialize=0 )

def con_money_inside_TID_1(model, tid, date):

    if date == 0:
        out = model.money_in_dict[(tid, date)]
    else:
        out = model.money_inside_TID[tid, date - 1] + model.money_in_dict[(tid, date)]


    return model.money_inside_TID[tid, date] <= out

model.days_from_inc = pyo.Var(TIDS, DATES,  within=pyo.NonNegativeIntegers, initialize=0 )


def con_money_inside_TID_2(model, tid, date):
    return model.money_inside_TID[tid, date] <= (1 - model.money_inc[tid, date]) * M


model.con_money_inside_TID_2 = pyo.Constraint(TIDS, DATES, rule=con_money_inside_TID_2)


# Можно не водить переменные, а просто ограничиться ограничением
def con_days_from_inc(model, tid, date):

    if date == 0:
        out = days_from_inc_dict[tid]
    else:
        out = model.days_from_inc[tid, date - 1] + 1


    return model.days_from_inc[tid, date] <= out



model.con_days_from_inc = pyo.Constraint(TIDS, DATES, rule=con_days_from_inc)


def con_max_money(model, tid, date):
    return model.money_inside_TID[tid, date] <= model.MAX_MONEY


model.con_max_money = pyo.Constraint(TIDS, DATES, rule=con_max_money)


def con_max_days_inc(model, tid, date):
    return model.money_inside_TID[tid, date] <= model.MAX_DAYS_INC


model.con_max_days_inc = pyo.Constraint(TIDS, DATES, rule=con_max_days_inc)


def costs_from_money(model, tid, date):
    return model.money_inside_TID[tid, date] * model.OVERNIGHT_BY_DAY


model.costs_from_money = pyo.Expression(TIDS, DATES, rule=costs_from_money )






model.OBJ = pyo.Objective(expr=quicksum([-1 * model.money_inside_TID[tid, date] for tid, date in itertools.product(TIDS, DATES) ]) +
                                quicksum([-1 * model.days_from_inc[tid, date] for tid, date in itertools.product(TIDS, DATES) ]) +
                                quicksum([-1 * model.costs_from_money[tid, date] for tid, date in itertools.product(TIDS, DATES) ]), sense=pyo.minimize)
# minimize

# , executable="/usr/local/Cellar/cbc/2.10.8/bin/cbc"
opt = SolverFactory('cbc')

for key in kwargs:
    opt.options[key] = kwargs[key]

results = opt.solve(model)

results

{'Problem': [{'Name': 'unknown', 'Lower bound': -58.00306849, 'Upper bound': -58.00306849, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 0, 'Number of binary variables': 4, 'Number of integer variables': 8, 'Number of nonzeros': 0, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.0, 'Wallclock time': 0.01, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}, 'Black box': {'Number of iterations': 0}}, 'Error rc': 0, 'Time': 0.04576396942138672}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

{'Problem': [{'Name': 'unknown', 'Lower bound': -80.0, 'Upper bound': -80.0, 'Number of objectives': 1, 'Number of constraints': 2, 'Number of variables': 4, 'Number of binary variables': 4, 'Number of integer variables': 4, 'Number of nonzeros': 4, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.0, 'Wallclock time': 0.01, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}, 'Black box': {'Number of iterations': 0}}, 'Error rc': 0, 'Time': 0.07807803153991699}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [ ]:

# BIG_NUM = 40*60
# postomat_places = list(fixed_points) + list(possible_postomats)
# center_mass_set = set(distances['id_center_mass'].to_list())
# metro_points_set = set(distanses_metro['object_id_metro'].to_list())

# distances_dict = {(id_center_mass, postomat_place_id): walk_time for _, postomat_place_id, id_center_mass , _, _, walk_time in distances.itertuples()}
# distances_metro_dict = {(object_id_metro, object_id): walk_time for _, object_id, object_id_metro ,  _, walk_time in distanses_metro.itertuples()}

# Создание конкретной модели pyomo
model = pyo.ConcreteModel()

# Переменные
model.has_postomat = pyo.Var(postomat_places, within=pyo.Binary, initialize=0)

if precalculated_points is not None:
    for point in precalculated_points:
        model.has_postomat[point] = 1


for fixed_point in fixed_points:
    model.has_postomat[fixed_point].fix(1)

model.center_mass_time_to_nearest_postamat = pyo.Var(population_points, within=pyo.NonNegativeReals)

#Ограничения

def con_center_mass_time_to_nearest_postamat(model, *data):
    _, id_center_mass, postomat_place_id = data
    return model.center_mass_time_to_nearest_postamat[id_center_mass] >= distances_dict[(id_center_mass, postomat_place_id)] * model.has_postomat[postomat_place_id]

model.con_center_mass_time_to_nearest_postamat = pyo.Constraint( list(distances[['id_center_mass',	'object_id']].itertuples()) ,rule=con_center_mass_time_to_nearest_postamat)

def center_mass_has_postomat(model, center_mass_id):
    only_needed_dist = distances.loc[distances['id_center_mass'] == center_mass_id]
    out = 0
    for object_id in only_needed_dist['object_id']:
        out += model.has_postomat[object_id]
    return out


model.center_mass_has_postomat = pyo.Expression(population_points, rule=center_mass_has_postomat )



def con_center_mass_has_postomat(model, center_mass_id):
    return model.center_mass_time_to_nearest_postamat[center_mass_id] >= (1 - model.center_mass_has_postomat[center_mass_id]) * BIG_NUM 


model.con_center_mass_has_postomat = pyo.Constraint(population_points, rule=con_center_mass_has_postomat)



model.metro_time_to_nearest_postamat = pyo.Var(object_id_metro_list, within=pyo.NonNegativeReals)

def con_metro_time_to_nearest_postamat(model, *data):
    _, object_id_metro, postomat_place_id = data
    return model.metro_time_to_nearest_postamat[object_id_metro] >= distances_metro_dict[(object_id_metro, postomat_place_id)] * model.has_postomat[postomat_place_id]

model.con_metro_time_to_nearest_postamat = pyo.Constraint( list(distanses_metro[['object_id_metro',	'object_id']].itertuples()) ,rule=con_metro_time_to_nearest_postamat)


def metro_has_postomat(model, metro_id):
    only_needed_dist = distanses_metro.loc[distanses_metro['object_id_metro'] == metro_id]
    out = 0
    for object_id in only_needed_dist['object_id']:
        out += model.has_postomat[object_id]
    return out


model.metro_has_postomat = pyo.Expression(object_id_metro_list, rule=metro_has_postomat )

def con_metro_has_postomat(model, metro_id):
    return model.metro_time_to_nearest_postamat[metro_id] >= (1 - model.metro_has_postomat[metro_id]) * BIG_NUM 


model.con_metro_has_postomat = pyo.Constraint(object_id_metro_list, rule=con_metro_has_postomat)


model.needed_postamats = pyo.Constraint(expr=sum([model.has_postomat[p] for  p in postomat_places]) <= quantity_postamats_to_place)

sum_center_mass = sum(model.center_mass_time_to_nearest_postamat[p] * population_dict[p] for p in population_points) 
sum_metro = sum(model.metro_time_to_nearest_postamat[p] * population_dict[p] for p in object_id_metro_list)
# # Целевая
model.OBJ = pyo.Objective(expr=((1 - metro_weight) * sum_center_mass + (metro_weight) * sum_metro), sense=pyo.minimize)
# minimize

# , executable="/usr/local/Cellar/cbc/2.10.8/bin/cbc"
opt = SolverFactory('cbc')

for key in kwargs:
    opt.options[key] = kwargs[key]

results = opt.solve(model)


optimised_list = get_chosen_postomats(model)

optimised_list_no_fixed = list(set(optimised_list).difference(set(fixed_points)))